In [6]:
import pandas
from arcgis.features import GeoAccessor

In [53]:
#File paths
stem_path = r'C:/Users/tjjoh/Documents/GIS5571'

gdb_path = stem_path + '/FinalProject_GDB.gdb'
fc_path = gdb_path + r'/Weather'

weather_noon_path = gdb_path + r'/Weather1200'

idw1_path = gdb_path + r'/IDW1'
idw2_path = gdb_path + r'/IDW2'
natneigh_path = gdb_path + r'/natneigh'
spline_path = gdb_path + r'/spline'
kriging_path = gdb_path + r'/kriging'

In [19]:
#Create new file geodatabase
arcpy.management.CreateFileGDB(
    out_folder_path = stem_path,
    out_name = 'FinalProject_GDB',
    out_version = 'CURRENT'
)

<Result 'C:/Users/tjjoh/Documents/GIS5571/FinalProject_GDB.gdb'>

In [55]:
#URL to retrieve weather data from NDAWN
url = 'https://ndawn.ndsu.nodak.edu/table.csv?station>0&variable=hdt&ttype=hourly&quick_pick=&begin_date=2024-03-25&end_date=2024-03-25'

In [56]:
#Read data into a pandas dataframe
weather = pandas.read_csv(url, skiprows = [0,1,2,4])

In [57]:
#Convert data to a spatially-enabled dataframe
weather_sedf = GeoAccessor.from_xy(weather, x_column = 'Longitude', y_column = 'Latitude', sr = 4326)

In [58]:
#Convert spatially-enabled dataframe into a feature class
weather_fc = weather_sedf.spatial.to_featureclass(location = fc_path)

In [59]:
#Create a new temporary feature layer
arcpy.management.MakeFeatureLayer(
    in_features = weather_fc,
    out_layer = 'temp_layer'
)

<Result 'temp_layer'>

In [60]:
#Select only data point for the 12:00 hour
arcpy.management.SelectLayerByAttribute(
    in_layer_or_view = 'temp_layer',
    selection_type = 'NEW_SELECTION',
    where_clause = 'hour = 1200'
)

<Result 'temp_layer'>

In [61]:
#Copy features from the temporary layer to a new feature class
arcpy.management.CopyFeatures(
    in_features = 'temp_layer',
    out_feature_class = weather_noon_path
)

<Result 'C:\\Users\\tjjoh\\Documents\\GIS5571\\FinalProject_GDB.gdb\\Weather1200'>

In [62]:
#Delete temporary layer
arcpy.management.Delete(
    in_data = 'temp_layer'
)

<Result 'true'>

**Deterministic Interpolation Methods**

In [63]:
#First-order inverse distance weighting interpolation
with arcpy.EnvManager(scratchWorkspace = gdb_path):
    IDW1 = arcpy.sa.Idw(
        in_point_features = weather_noon_path,
        z_field = 'avg_air_temp',
        cell_size = 0.021010908,
        power = 1,
        search_radius = 'VARIABLE 12',
        in_barrier_polyline_features = None
    )
    IDW1.save(idw1_path)

In [64]:
#Second-order inverse distance weighting interpolation
with arcpy.EnvManager(scratchWorkspace = gdb_path):
    IDW2 = arcpy.sa.Idw(
        in_point_features = weather_noon_path,
        z_field = 'avg_air_temp',
        cell_size = 0.021010908,
        power = 2,
        search_radius = 'VARIABLE 12',
        in_barrier_polyline_features = None
    )
    IDW2.save(idw2_path)

In [65]:
#Natural neighbor interpolation
with arcpy.EnvManager(scratchWorkspace = gdb_path):
    NatNeigh = arcpy.sa.NaturalNeighbor(
        in_point_features = weather_noon_path,
        z_field = 'avg_air_temp',
        cell_size = 0.021010908
    )
    NatNeigh.save(natneigh_path)

In [66]:
#Spline interpolation
with arcpy.EnvManager(scratchWorkspace = gdb_path):
    Spline = arcpy.sa.Spline(
        in_point_features = weather_noon_path,
        z_field = 'avg_air_temp',
        cell_size = 0.021010908,
        spline_type = 'REGULARIZED',
        weight = 0.1,
        number_points = 12
    )
    Spline.save(spline_path)

**Stochastic Interpolation Methods**

In [67]:
with arcpy.EnvManager(scratchWorkspace = gdb_path):
    Krig = arcpy.sa.Kriging(
        in_point_features = weather_noon_path,
        z_field = 'avg_air_temp',
        kriging_model = 'Spherical # # # #',
        cell_size = 0.021010908,
        search_radius = 'VARIABLE 12',
        out_variance_prediction_raster = None
    )
    Krig.save(kriging_path)